In [62]:
import sys
sys.path.append('../')
from ftx_utilities import *
%matplotlib inline

In [110]:
concentration_limit=0.5
slippage_override=2e-4
sharpe=lambda x: np.mean(x)/np.std(x)
sortino=lambda x: np.mean(x)/np.std(x[x<0])
drawdown=lambda x: (x.cumsum()-x.cumsum().cummax()).cummin().loc[x.index[-1]]

In [82]:
# basics
def read_it(filename):
    df=pd.read_pickle(filename)
    #df[df.select_dtypes(include=['number']).columns] *= 100/1e5
    df['totalPnL']=(df['RealizedCarry']+df['transactionCost'])
    df['trackingError']=df['RealizedCarry']-df['ExpectedCarry']
    df['track2']=df['trackingError']*df['trackingError']
    df['rel2']=df['RealizedCarry']*df['RealizedCarry']
    df=df[
        (df['concentration_limit']==concentration_limit)
        &(df['slippage_override']==slippage_override)]
    return df.drop(['concentration_limit','slippage_override'],axis=1)

## benchmark

In [83]:
# read benchmark
df=read_it('archived data/benchmark_blind.pickle')
mdf=df.set_index(['name','time']) \
        .unstack('name') \
            .reorder_levels([1,0],axis=1)
total=mdf.xs('total',level='name',axis=1)

In [103]:
perfs=df.groupby(by=['name'])['absWeight'].describe().sort_values(by='mean',key=abs,ascending=False).head(50)
mains=df[df['name'].isin(list(perfs.index)+['total','USD'])]

In [104]:
stats=mains.groupby(by=['name'])[['RealizedCarry','transactionCost','trackingError','optimalWeight']].agg(['mean','std'])*100/1e5
stats[('relError','')]=np.sqrt((np.power(stats[('trackingError','mean')],2)+np.power(stats[('trackingError','std')],2))/(np.power(stats[('RealizedCarry','mean')],2)+np.power(stats[('RealizedCarry','std')],2)))
stats.sort_values(by=('transactionCost','mean'))

field      RealizedCarry            transactionCost            trackingError  \
                    mean        std            mean        std          mean   
name                                                                           
total          11.957975  29.118401       -2.233412  15.436246     -5.554269   
BAND-PERP       1.070471   3.456323       -0.185530   1.725056     -0.481433   
BNT-PERP        2.206650   5.584825       -0.175211   1.667985     -0.193035   
GRT-PERP        1.018647   5.298776       -0.172107   1.384010     -0.707995   
CEL-PERP        0.351741   2.110599       -0.171797   1.675629     -0.322362   
OKB-PERP        0.498579   5.401724       -0.154868   1.007269     -0.864403   
ALPHA-PERP      1.307486   4.197603       -0.149247   1.281565     -0.299225   
MKR-PERP        0.864402   3.903223       -0.107731   0.842979     -0.515108   
RUNE-PERP       0.146358   2.226027       -0.106104   0.670877     -0.428024   
1INCH-PERP      0.201135   1.098616       -0.091144   0.942377     -0.152761   
SOL-PERP        0.494082   1.430319       -0.085617   0.845517     -0.037242   
SNX-PERP        0.165246   1.496811       -0.080873   0.625554     -0.225222   
KNC-PERP        0.008853   1.073164       -0.078278   0.938279     -0.068171   
SUSHI-PERP      0.777038   2.919708       -0.068813   0.585551     -0.132695   
SXP-PERP        0.919867   4.428452       -0.067490   0.613679     -0.451248   
AAVE-PERP       0.351855   1.216270       -0.066232   0.711547     -0.094454   
DOGE-PERP       0.947157   3.250705       -0.058793   0.455086     -0.203100   
BNB-PERP        0.099951   0.628702       -0.053632   0.637414     -0.035218   
TRX-PERP        0.014262   0.306234       -0.046594   0.740197     -0.037334   
MATIC-PERP      0.769392   2.827683       -0.041635   0.453532     -0.072603   
UNI-PERP        0.464727   1.810935       -0.040970   0.307755     -0.145204   
BCH-PERP        0.213750   0.991270       -0.036486   0.515555     -0.025135   
LINK-PERP       0.484463   1.505833       -0.033765   0.263681     -0.050325   
BTC-PERP        0.015247   0.106186       -0.030827   0.424595     -0.005815   
ETH-PERP        0.002068   0.032790       -0.029120   0.475649     -0.001478   
OMG-PERP        0.188115   0.881272       -0.027276   0.262794     -0.045542   
YFI-PERP        0.070970   0.470570       -0.026782   0.238780     -0.075129   
XRP-PERP        0.062068   0.424889       -0.019009   0.154559     -0.033590   
LTC-PERP        0.001743   0.049153       -0.018010   0.261063     -0.003116   
USDT-PERP      -0.014283   0.182944       -0.009470   0.118871     -0.005077   
USD            -1.744065   6.481155        0.000000   0.000000      0.157775   

field                 optimalWeight                relError  
                  std          mean           std            
name                                                         
total       23.487492    100.000000  1.565228e-14  0.766731  
BAND-PERP    3.090743      1.867175  1.418036e+01  0.864499  
BNT-PERP     4.474792     -9.514124  1.740570e+01  0.745875  
GRT-PERP     4.985709     -1.099992  1.626367e+01  0.933268  
CEL-PERP     1.999078     -0.336428  1.180949e+01  0.946345  
OKB-PERP     6.549914     -3.483299  1.305030e+01  1.217897  
ALPHA-PERP   3.251013      6.649365  1.484831e+01  0.742577  
MKR-PERP     3.360249      3.727859  1.297634e+01  0.850345  
RUNE-PERP    2.556997      1.389112  7.181873e+00  1.162155  
1INCH-PERP   1.286795      0.230173  4.686551e+00  1.160228  
SOL-PERP     0.886710      3.465820  8.361689e+00  0.586480  
SNX-PERP     1.666456     -0.287940  7.386475e+00  1.116676  
KNC-PERP     1.282407     -0.827323  5.075429e+00  1.196624  
SUSHI-PERP   2.253329      3.839244  1.116141e+01  0.747097  
SXP-PERP     3.957597      4.500765  1.255274e+01  0.880667  
AAVE-PERP    0.902992      1.934969  5.926146e+00  0.717075  
DOGE-PERP    2.153759      4.764563  1.332373e+01  0.638922  
BNB-PERP     0.503672      0.084836  3.

In [105]:
(mains.pivot_table(index='time',columns='name',values='totalPnL',aggfunc='mean')/24/365.25).iplot()

## runs

In [ ]:
# read full run
df=read_it('archived data/full_run.pickle')
mdf=df.set_index(['signal_horizon','holding_period','name','time']) \
        .unstack(['signal_horizon','holding_period','name']) \
            .reorder_levels([1,2,3,0],axis=1)
total=mdf.xs(('totalPnL','total'),level=('field','name'),axis=1)*100/1e5

In [108]:
# sharpe is bad
total.apply(lambda f: sharpe(f.dropna()),axis=0).unstack()

holding_period,0 days 06:00:00,0 days 12:00:00,1 days 00:00:00,2 days 00:00:00,3 days 00:00:00,4 days 00:00:00,5 days 00:00:00
signal_horizon,,,,,,,
0 days 12:00:00,-0.058615,0.491320,NaN,NaN,NaN,NaN,NaN
1 days 00:00:00,-0.058208,0.294010,0.924270,NaN,NaN,NaN,NaN
2 days 00:00:00,-0.054421,0.429306,0.997867,1.173736,NaN,NaN,NaN
3 days 00:00:00,-0.061333,0.305493,0.902892,1.118022,1.286722,NaN,NaN
4 days 00:00:00,-0.059642,0.260735,0.846793,1.094914,1.260790,1.312444,NaN
5 days 00:00:00,-0.047017,0.178943,0.702489,1.190681,1.167614,1.341646,1.551580
7 days 00:00:00,-0.059483,0.102340,0.884217,1.148275,1.197592,1.276588,1.253307
10 days 00:00:00,-0.058636,-0.083802,0.824453,1.028955,1.016925,1.120752,1.202375
30 days 00:00:00,-0.050424,-0.098218,0.298392,0.590868,0.707584,0.875428,0.853092


In [111]:
# sortino is great
total.apply(lambda f: sortino(f.dropna()),axis=0).unstack()

C:\Users\david\AppData\Local\Temp/ipykernel_9040/14136895.py:4: RuntimeWarning:

divide by zero encountered in double_scalars



holding_period,0 days 06:00:00,0 days 12:00:00,1 days 00:00:00,2 days 00:00:00,3 days 00:00:00,4 days 00:00:00,5 days 00:00:00
signal_horizon,,,,,,,
0 days 12:00:00,-0.027135,0.523267,NaN,NaN,NaN,NaN,NaN
1 days 00:00:00,-0.023707,0.320548,1.411857,NaN,NaN,NaN,NaN
2 days 00:00:00,-0.021737,0.463662,1.981812,3.319196,NaN,NaN,NaN
3 days 00:00:00,-0.024642,0.232153,1.485113,3.239701,inf,NaN,NaN
4 days 00:00:00,-0.024294,0.165247,1.190225,3.241436,inf,29.054783,NaN
5 days 00:00:00,-0.022217,0.134381,1.211995,4.692626,2.943249,14.781833,inf
7 days 00:00:00,-0.031417,0.064155,2.374393,9.885021,inf,inf,NaN
10 days 00:00:00,-0.029024,-0.051797,2.213880,3.723148,5.822618,44.228750,14.343324
30 days 00:00:00,-0.025771,-0.056541,1.110627,2.220044,1.972808,4.927057,4.458588


In [109]:
# drawdown is great
total.apply(lambda f: drawdown(f.dropna()),axis=0).unstack()*100/1e5

holding_period,0 days 06:00:00,0 days 12:00:00,1 days 00:00:00,2 days 00:00:00,3 days 00:00:00,4 days 00:00:00,5 days 00:00:00
signal_horizon,,,,,,,
0 days 12:00:00,-0.057411,-0.129161,NaN,NaN,NaN,NaN,NaN
1 days 00:00:00,-0.039523,-0.228705,-0.171751,NaN,NaN,NaN,NaN
2 days 00:00:00,-0.016773,-0.170500,-0.108548,-0.059572,NaN,NaN,NaN
3 days 00:00:00,-0.047709,-0.167526,-0.087300,-0.017472,-0.017107,NaN,NaN
4 days 00:00:00,-0.049605,-0.196519,-0.081449,-0.048484,-0.007983,-0.007112,NaN
5 days 00:00:00,-0.064112,-0.175991,-0.088402,-0.022577,-0.030746,-0.010342,-0.001248
7 days 00:00:00,-0.025634,-0.048166,-0.091254,-0.016584,-0.012549,-0.001047,0.000000
10 days 00:00:00,-0.011592,-0.078324,-0.107923,-0.028940,-0.018394,-0.005421,-0.010881
30 days 00:00:00,-0.009837,-0.010798,-0.178183,-0.093025,-0.063610,-0.050543,-0.034553


In [92]:
total.xs([timedelta(days=2),timedelta(days=7)],level=['holding_period','signal_horizon'],axis=1).dropna().cumsum().iplot()


C:\Users\david\AppData\Local\Temp/ipykernel_9040/3353639744.py:1: FutureWarning:

Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.



## todo
# - tx cost = 0 for benchmark, 5bps
# - realized vs expected fit -> finer hold/sig
# - raise the bar for volume, blowup
# - munraj

## Comparative run statistics

In [ ]:
runs=compile_runs()
runs

In [ ]:
c=0.5
all=1*a.xs([c,'RealizedCarry'],
       level=['concentration_limit','field'])\
    +a.xs([c,'transactionCost'],
       level=['concentration_limit','field'])
all['total'].groupby(by=['holding_period','signal_horizon']).describe()*100

## benchmark: transaction costs block rebalance ?

In [ ]:
benchmark=a.xs([c,timedelta(hours=1),timedelta(hours=1)],
       level=['concentration_limit','holding_period','signal_horizon']).reorder_levels(['field','time'])

## Zoom on a run

In [ ]:
run=a.xs([c,timedelta(days=2),timedelta(days=7)],
       level=['concentration_limit','holding_period','signal_horizon']).reorder_levels(['field','time'])

In [ ]:
relevant=(run.T['absWeight'].T).max().sort_values(ascending=False)
relevant=relevant[relevant>0.02]
run=run[relevant.index]

In [ ]:
fig=px.line(run.loc['optimalWeight'])
fig.show()

In [ ]:
fig=px.line(run.loc['RealizedCarry'])
fig.show()

In [ ]:
fig=px.line(run.loc['ExpectedCarry'])
fig.show()